__Imports__

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

__Read in Data__

In [ ]:
import cudf
print('RAPIDS version',cudf.__version__)

In [2]:
train = pd.read_csv("C:/Users/Michael/Github/transactions_train.csv")

In [3]:
# train = pd.DataFrame(train_1[:1000])

In [4]:
train

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2
...,...,...,...,...,...
31788319,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,929511001,0.059305,2
31788320,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,891322004,0.042356,2
31788321,2020-09-22,fff380805474b287b05cb2a7507b9a013482f7dd0bce0e...,918325001,0.043203,1
31788322,2020-09-22,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,833459002,0.006763,1


In [5]:
max_length = len(np.unique(train['article_id']))

__Get the sequences of articles together by customer__

In [6]:
train['article_id'] = train['article_id'].astype(str)
train['article_id'] = train.groupby(['customer_id'])['article_id'].transform(lambda x: ','.join(x))

In [7]:
train

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,"663713001,541518023,663713001,578020002,723529...",0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,"663713001,541518023,663713001,578020002,723529...",0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,"505221004,685687003,685687004,685687001,505221...",0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,"505221004,685687003,685687004,685687001,505221...",0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,"505221004,685687003,685687004,685687001,505221...",0.016932,2
...,...,...,...,...,...
31788319,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,"650037001,688688002,507910001,652000008,626810...",0.059305,2
31788320,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,"650037001,688688002,507910001,652000008,626810...",0.042356,2
31788321,2020-09-22,fff380805474b287b05cb2a7507b9a013482f7dd0bce0e...,"129085001,855077001,861023001,861023001,733098...",0.043203,1
31788322,2020-09-22,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,"675068002,697564010,697564010,742924002,699081...",0.006763,1


In [8]:
train = train.groupby(['t_dat','customer_id']).head()

In [9]:
train.drop(['price','sales_channel_id'],axis=1,inplace=True)

C:\Users\Michael\AppData\Local\Temp\ipykernel_12224\3742703963.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop(['price','sales_channel_id'],axis=1,inplace=True)


In [10]:
train = pd.DataFrame(train.drop_duplicates())

In [11]:
train

,t_dat,customer_id,article_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,"663713001,541518023,663713001,578020002,723529..."
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,"505221004,685687003,685687004,685687001,505221..."
7,2018-09-20,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,"688873012,501323011,598859003,688873020,688873..."
12,2018-09-20,0008968c0d451dbc5a9968da03196fe20051965edde741...,"531310002,529841001,662980002,496111003,561752..."
14,2018-09-20,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,"501820043,501820043,674681001,671505001,671505..."
...,...,...,...
31788310,2020-09-22,ffd4cf2217de4a0a3f9f610cdec334c803692a18af08ac...,"507909022,507909004,507909004,633130002,687934..."
31788319,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,"650037001,688688002,507910001,652000008,626810..."
31788321,2020-09-22,fff380805474b287b05cb2a7507b9a013482f7dd0bce0e...,"129085001,855077001,861023001,861023001,733098..."
31788322,2020-09-22,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,"675068002,697564010,697564010,742924002,699081..."


In [12]:
def date_master(article_length):
    length = len(article_length.split(","))
    return length

In [13]:
def date_multiplier(dt,length):
    date = dt
    temp = []
    for i in range(int(length)):
        temp.append(date)
    return ",".join(temp)

In [14]:
train['length'] = train.apply(lambda x : date_master(x['article_id']),axis=1)

In [15]:
train['t_dat'] = pd.to_datetime(train['t_dat'])

In [16]:
train = train.sort_values(['customer_id','t_dat'])

In [17]:
train['nb_days'] = train.groupby('customer_id')['t_dat'].apply(lambda x: x-x.iat[0]).dt.days

In [18]:
train['nb_days'] = train.apply(lambda x : date_multiplier(str(x['nb_days']),x['length']),axis=1)

In [19]:
train['t_dat'] = train['t_dat'].astype(str)
train['t_dat'] = train.groupby(['customer_id'])['t_dat'].transform(lambda x: ' '.join(x))

In [20]:
train.groupby(['customer_id']).head()

,t_dat,customer_id,article_id,length,nb_days
4212358,2018-12-27 2019-05-02 2019-05-25 2019-07-25 20...,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,"625548001,176209023,627759010,697138006,568601...",21,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0"
9663224,2018-12-27 2019-05-02 2019-05-25 2019-07-25 20...,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,"625548001,176209023,627759010,697138006,568601...",21,"126,126,126,126,126,126,126,126,126,126,126,12..."
10754876,2018-12-27 2019-05-02 2019-05-25 2019-07-25 20...,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,"625548001,176209023,627759010,697138006,568601...",21,"149,149,149,149,149,149,149,149,149,149,149,14..."
14479578,2018-12-27 2019-05-02 2019-05-25 2019-07-25 20...,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,"625548001,176209023,627759010,697138006,568601...",21,"210,210,210,210,210,210,210,210,210,210,210,21..."
16728347,2018-12-27 2019-05-02 2019-05-25 2019-07-25 20...,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,"625548001,176209023,627759010,697138006,568601...",21,"265,265,265,265,265,265,265,265,265,265,265,26..."
...,...,...,...,...,...
24185205,2020-04-05 2020-04-09 2020-04-25 2020-06-22,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,"821395003,806241002,714790020,866755002,840360...",7,"0,0,0,0,0,0,0"
24375394,2020-04-05 2020-04-09 2020-04-25 2020-06-22,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,"821395003,806241002,714790020,866755002,840360...",7,"4,4,4,4,4,4,4"
25077914,2020-04-05 2020-04-09 2020-04-25 2020-06-22,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,"821395003,806241002,714790020,866755002,840360...",7,"20,20,20,20,20,20,20"
27806865,2020-04-05 2020-04-09 2020-04-25 2020-06-22,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,"821395003,806241002,714790020,866755002,840360...",7,"78,78,78,78,78,78,78"


In [21]:
train.drop(['t_dat','length'],axis=1,inplace=True)

In [22]:
train.head()

,customer_id,article_id,nb_days
4212358,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,"625548001,176209023,627759010,697138006,568601...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0"
9663224,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,"625548001,176209023,627759010,697138006,568601...","126,126,126,126,126,126,126,126,126,126,126,12..."
10754876,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,"625548001,176209023,627759010,697138006,568601...","149,149,149,149,149,149,149,149,149,149,149,14..."
14479578,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,"625548001,176209023,627759010,697138006,568601...","210,210,210,210,210,210,210,210,210,210,210,21..."
16728347,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,"625548001,176209023,627759010,697138006,568601...","265,265,265,265,265,265,265,265,265,265,265,26..."


## Feature Transformation

__Tokenization & Encoding of item_id__

In [23]:
import tensorflow as tf
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=12,split=',')

In [24]:
tokenizer.fit_on_texts(train['article_id'])

In [25]:
tokenizer.sequences_to_texts([[1,2,3,3,4,5,6,7,8,9,10,11,12]])

['706016001 706016002 610776002 610776002 372860001 759871002 610776001 673677002 399223001 156231001 372860002 448509014']

In [26]:
import numpy as np
bins = np.linspace(1,100,100)
def returning_binning(arr):
    temp = []
    arr = arr.split(',')
    for i in arr:
        temp.append(str(np.digitize(i,bins)))
    return ",".join(temp)

In [30]:
train['nb_days'] = train.nb_days.apply(lambda x: returning_binning(x))


KeyboardInterrupt



In [85]:
temp

['0',
 '4',
 '158',
 '339',
 '0',
 '0',
 '58',
 '0',
 '17',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '150']

In [90]:
funct = lambda x: ','.join(x)

In [91]:
funct(temp)

'0,4,158,339,0,0,58,0,17,0,0,0,0,0,0,0,150'

__Converting Hex to Integers__

In [39]:
def hex_to_signed(source):
    """Convert a string hex value to a signed hexidecimal value.

    This assumes that source is the proper length, and the sign bit
    is the first bit in the first byte of the correct length.

    hex_to_signed("F") should return -1.
    hex_to_signed("0F") should return 15.
    """
    if not isinstance(source, str):
        raise ValueError("string type required")
    if 0 == len(source):
        raise valueError("string is empty")
    sign_bit_mask = 1 << (len(source)*4-1)
    other_bits_mask = sign_bit_mask - 1
    value = int(source, 16)
    return -(value & sign_bit_mask) | (value & other_bits_mask)

__Selecting and formating data types in dataframe__

In [42]:
train['customer_id'] = train['customer_id'].apply(lambda x : hex_to_signed(x))
#train['article_id'] = train.article_id.astype('int32')
train['t_dat'] = pd.to_datetime(train['t_dat'])
train = train[['t_dat','customer_id','article_id']]
train.head()

,t_dat,customer_id,article_id
0,2018-09-20,6116990065284462810764946559699987067290722768...,663713001 541518023 663713001 578020002 723529...
1,2018-09-20,6116990065284462810764946559699987067290722768...,663713001 541518023 663713001 578020002 723529...
2,2018-09-20,8639559392216894130813047803836240585733025821...,505221004 685687003 685687004 685687001 505221...
3,2018-09-20,8639559392216894130813047803836240585733025821...,505221004 685687003 685687004 685687001 505221...
4,2018-09-20,8639559392216894130813047803836240585733025821...,505221004 685687003 685687004 685687001 505221...


__Saving this for later__

In [7]:
# another_train = train

__Customer's Last Week Purchases__

In [43]:
tmp = train.groupby('customer_id').t_dat.max().reset_index()
tmp.columns = ['customer_id','max_dat']
train = train.merge(tmp,on=['customer_id'],how='left')
train['diff_dat'] = (train.max_dat - train.t_dat).dt.days
train = train.loc[train['diff_dat']<=6]
print('Train shape:',train.shape)
train.head()

Train shape: (5181535, 5)


,t_dat,customer_id,article_id,max_dat,diff_dat
49,2018-09-20,5981405501434810419031299841272056822028636268...,649356002 579941002 629760002 625229004,2018-09-20,0
50,2018-09-20,5981405501434810419031299841272056822028636268...,649356002 579941002 629760002 625229004,2018-09-20,0
51,2018-09-20,5981405501434810419031299841272056822028636268...,649356002 579941002 629760002 625229004,2018-09-20,0
52,2018-09-20,5981405501434810419031299841272056822028636268...,649356002 579941002 629760002 625229004,2018-09-20,0
61,2018-09-20,1132591264840105692788114398749857557049430095...,613456009 633675001 648719001 427114015 567475...,2018-09-20,0


__Recommendation Most Often Prev Purchased Items__

In [9]:
tmp = train.groupby(['customer_id','article_id'])['t_dat'].agg('count').reset_index()
tmp.columns = ['customer_id','article_id','ct']
train = train.merge(tmp,on=['customer_id','article_id'],how='left')
train = train.sort_values(['ct','t_dat'],ascending=False)
train = train.drop_duplicates(['customer_id','article_id'])
train = train.sort_values(['ct','t_dat'],ascending=False)
train.head()

,t_dat,customer_id,article_id,max_dat,diff_dat,ct
1158548,2019-07-16,1122880311838375530037224418800749068125981861...,719348003,2019-07-16,0,100
80218,2018-10-04,-673191760722678788667587928976895833688046975...,557247001,2018-10-04,0,86
2149868,2020-03-06,9249039390400736591087536058610073983058079884...,852521001,2020-03-06,0,81
3423153,2020-07-06,-518177034797192986786595171574626635942011784...,685813001,2020-07-06,0,80
871769,2019-05-14,1283729545703826238940687917472870211201319841...,695545001,2019-05-14,0,80


In [10]:
df = train[['customer_id','article_id']]

__Recommend Items Purchased Together__

# USE PANDAS TO MAP COLUMN WITH DICTIONARY

In [ ]:
pairs = np.load('pairs.npy',allow_pickle=True).item()
train['article_id2'] = train.article_id.map(pairs)
train.head()

# RECOMMENDATION OF PAIRED ITEMS

In [ ]:
train2 = train[['customer_id','article_id2']].copy()
train2 = train2.loc[train2.article_id2.notnull()]
train2 = train2.drop_duplicates(['customer_id','article_id2'])
train2 = train2.rename({'article_id2':'article_id'},axis=1)
train2.head()

# CONCATENATE PAIRED ITEM RECOMMENDATION AFTER PREVIOUS PURCHASED RECOMMENDATIONS

In [ ]:
train = train[['customer_id','article_id']]
train = pd.concat([train,train2],axis=0,ignore_index=True)
train.article_id = train.article_id.astype('int32')
train = train.drop_duplicates(['customer_id','article_id'])
train.head()

# CONVERT RECOMMENDATIONS INTO SINGLE STRING

In [ ]:
train.article_id = ' 0' + train.article_id.astype('str')
preds = pd.DataFrame( train.groupby('customer_id').article_id.sum().reset_index() )
preds.columns = ['customer_id','prediction']
preds.head()

In [ ]:
another_train.t_dat = pd.to_datetime(another_train.t_dat)
another_train = another_train.loc[another_train.t_dat >= pd.to_datetime('2020-09-16')]
top12 = ' 0' + ' 0'.join(another_train.article_id.value_counts().index.astype('str')[:12])
print("Last week's top 12 popular items:")
print( top12 )

In [ ]:
sub = pd.read_csv("C:/Users/Michael/Github/sample_submission.csv")
sub = sub[['customer_id']]
sub['customer_id_2'] = sub['customer_id'].apply(lambda x : hex_to_signed(x))
sub = sub.merge(preds.rename({'customer_id':'customer_id_2'},axis=1),on='customer_id_2', how='left').fillna('')

del sub['customer_id_2']
sub.prediction = sub.prediction + top12
sub.prediction = sub.prediction.str.strip()
sub.prediction = sub.prediction.str[:131]
#sub.to_csv(f'submission.csv',index=False)
sub.head()

In [ ]:
sub.to_csv(f'submission_04_30_2022.csv',index=False)